In [49]:
import timeit
import importlib
import relevancer as rlv
import pandas as pd
from sklearn.naive_bayes import MultinomialNB 
from bson.objectid import ObjectId
from collections import Counter
import numpy as np

pd.set_option("display.max_colwidth",200)


In [3]:
# this is just to load the latest version of the Relevancer after we edit the code.
importlib.reload(rlv)
active_col = "active_text"
rlv.set_active_column(active_col)


<module 'relevancer' from '/Users/alihurriyetoglu/Dropbox/Projects/Relevancer/relevancer.py'>

In [243]:
#importlib.reload(rlv)
my_token_pattern=r"[#@]?\w+\b|[\U00010000-\U0010ffff]"
rlvdb, rlvcl = rlv.connect_mongodb(configfile='elifdb.ini',coll_name="testcl")
# set active columns
active_col = "active_text"
rlv.set_active_column(active_col)

In [5]:
importlib.reload(rlv)

# after the first iteration, the annotated clusters should be excluded from the clustering.
# read this file from the tagged collection.
annotated_tw_ids = ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960'] #You should get the actual annotated tweet ids from the annotated tweets collection.
#annotated_tw_ids = []
# mongo_query=({'_id': {'$gte': begin, '$lte': end},'lang':'en'})
tweetlist = rlv.read_json_tweet_fields_database(rlvcl, mongo_query=({}), read_fields={'text': 1, 'id_str': 1, '_id': 0, 'user_id': 1}, tweet_count=-1, annotated_ids=annotated_tw_ids)#=tweetsDF)

rlv.logging.info("Number of tweets:" + str(len(tweetlist)))
print("Number of tweets:",len(tweetlist))

Number of tweets: 391649


In [185]:
tweetsDF = rlv.create_dataframe(tweetlist)
tweetsDF.head()

391649


,id_str,is_retweet,text,user_id
0,615329612881612800,False,RT @kararsizatom: Do you have an idea to stop this genocide? #StopTerrorisminChina @carlbildt http://t.co/edq1P8nTtw,1381676425
1,605474184714543104,False,RT @RumBegum: This situation won't go viral because it's Muslims being killed not Muslims killing #Rohingya #UNMustSaveBurmaMuslims http://…,2831033527
2,613073957319782400,False,"@eurireland The second GENOCIDE against #Biafrans as promised by #Buhari has begun, 3 days of unreported aerial Bombardment in #Biafraland",2188829916
3,613337519753555968,False,RT @WachukwuGodwin: @BBCScotlandNews #Buhari &amp; Nigeria declare war on #Biafra killing 100s of innocent Biafrans wit Napalm bombs GENOCIDE h…,1173943710
4,605477505651359744,False,#ModiMinistry Obama says Myanmar needs to end discrimination of Rohingya to succeed\n| Reuters http://t.co/zmagjFtMH8,2425268995


In [186]:
len(tweetsDF)

391649

In [187]:
#tweetsDF.to_pickle("20151005_tweetsDF_Genocide")

In [8]:
# Be careful do not overwrite tweetsDFBackUP with a modified tweetsDF
#tweetsDFBackUP = tweetsDF.copy()
#len(tweetsDFBackUP)

In [31]:
# Get a clean copy 
#importlib.reload(rlv)
tweetsDF = tweetsDFBackUP.copy()
len(tweetsDF)

391649

In [188]:
tweetsDF[active_col] = tweetsDF["text"].copy()
tweetsDF = rlv.tok_results(tweetsDF, elimrt = True)


Available attributes of the tweets: Index(['id_str', 'is_retweet', 'text', 'user_id', 'active_text', 'texttok'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 391649 entries, 0 to 391648
Data columns (total 6 columns):
id_str         391649 non-null object
is_retweet     391649 non-null bool
text           391649 non-null object
user_id        391649 non-null object
active_text    391649 non-null object
texttok        391649 non-null object
dtypes: bool(1), object(5)
memory usage: 18.3+ MB

tweet set summary: None
0                                RT @kararsizatom: Do you have an idea to stop this genocide? #StopTerrorisminChina @carlbildt http://t.co/edq1P8nTtw
1        RT @RumBegum: This situation won't go viral because it's Muslims being killed not Muslims killing #Rohingya #UNMustSaveBurmaMuslims http://…
2          @eurireland The second GENOCIDE against #Biafrans as promised by #Buhari has begun, 3 days of unreported aerial Bombardment in #Biafraland
3    RT @

/Users/alihurriyetoglu/Dropbox/Projects/Relevancer/relevancer.py:505: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  tweetsDF[active_column] = tweetsDF[tok_result_lower_col].copy()


In [189]:
len(tweetsDF)

175524

In [191]:
tweetsDF = rlv.normalize_text(tweetsDF)
len(tweetsDF), tweetsDF.columns

(175524,
 Index(['id_str', 'is_retweet', 'text', 'user_id', 'active_text', 'texttok',
        'is_notrt'],
       dtype='object'))

In [190]:
#importlib.reload(rlv)
#rlv.set_active_column(active_col)

In [194]:
def eliminate_duplicates_recursively(df, duplicate_elim_func):
            
        print("starting, length:",len(df))
        df = df.reindex(np.random.permutation(df.index))
        df.reset_index(inplace=True, drop=True)
        
        tmp_df2 = pd.DataFrame()
        for i in range(0, len(df), 10000):
            tmp_unique = duplicate_elim_func(df[i:i+step], similarity_threshold=0.20, debug=True, debug_threshold=10000)
            tmp_df2 = pd.concat([tmp_df2, tmp_unique], ignore_index=True)
            
        if len(df) > len(tmp_df2):
            print(str(len(df) - len(tmp_df2))+" tweets were eliminated!")
            return eliminate_duplicates_recursively(tmp_df2, duplicate_elim_func)
        
        return df

In [195]:
tweetsDF_uniq = eliminate_duplicates_recursively(tweetsDF.copy(), rlv.get_and_eliminate_near_duplicate_tweets)


starting, length: 175524
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6526 entries, 0 to 5565
Data columns (total 7 columns):
id_str         6526 non-null object
is_retweet     6526 non-null bool
text           6526 non-null object
user_id        6526 non-null object
active_text    6526 non-null object
texttok        6526 non-null object
is_notrt       6526 non-null bool
dtypes: bool(2), object(5)
memory usage: 318.7+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6633 entries, 10001 to 17328
Data columns (total 7 columns):
id_str         6633 non-null object
is_retweet     6633 non-null bool
text           6633 non-null object
user_id        6633 non-null object
active_text    6633 non-null object
texttok        6633 non-null object
is_notrt       6633 non-null bool
dtypes: bool(2), object(5)
memory usage: 323.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6636 entries, 20000 to 21924
Data columns (total 7 columns):
id_str         6636 non-null object
is_retweet   

In [196]:
len(tweetsDF_uniq)

77884

In [197]:
tweetsDF_uniq.to_pickle("20151005_unique_genocide_tweets.pickle")

In [198]:
tweetsDF_uniq[["active_text"]][:10]

,active_text
0,"why scilant world media, world humanity organisations on rohingya's jenucide &amp; holocast in myanmar by budhist???"
1,#rohingya need to be treated as #burmese citizens: #us - urlurlurl via usrusrusr #humanrights usrusrusr usrusrusr
2,usrusrusr usrusrusr allegation. that's nothing compared to the slaughter/ genocide of the pope and proven pedophile priests
3,#genocide in #gaza: viable #palestinian strategic options in the face of #israeli tactics -- puppet #masters -urlurlurl
4,#armenpress pope presented book on armenian genocide: the book “the armenian genocide: 100 years of s... urlurlurl #armenia
5,symbol encourages #genocide #racist #whitesboro seal ‘takes a little explaining’ urlurlurl via usrusrusr usrusrusr usrusrusr
6,turkish prime minister and his wife in tears after meeting rohingya in myanmar. urlurlurl via usrusrusr
7,"horrific pictures show huge suffering of the rohingya people, make me think it's all photoshopped because world silence is unbelievable."
8,"advance australia fair was written 1878, when genocide and our slave trade was in place...i don't want to sing any verse #singthesecondverse"
9,usrusrusr it's cool i'll let it slide this one time


In [224]:
importlib.reload(rlv)
rlv.set_active_column(active_col)

In [225]:
cluster_list = rlv.create_clusters(tweetsDF_uniq, my_token_pattern, min_dist_thres=0.725, max_dist_thres=0.875, min_max_diff_thres=0.4, nameprefix='1-', min_clusters=100, user_identifier='user_id')

In [226]:
print("Number of clusters:",len(cluster_list))
print("available cluster information:", cluster_list[0].keys())

Number of clusters: 145
available cluster information: dict_keys(['ctweettuplelist', 'twids', 'cnoprefix', 'rif', 'cno', 'cstr', 'user_entropy'])


In [227]:
i = 0

In [237]:
#i = i-2
print("No:",cluster_list[i]['cno'])
print("CStr:",cluster_list[i]['cstr'])
print("Cluster Tuple List:")
print(*[(c[0],c[2]) for c in cluster_list[i]['ctweettuplelist']], sep='\n')
# Add any field you want to observe.
i+=1

No: 324
CStr: cluster number and size are: 324    62

Cluster Tuple List:
(0.78476101036957813, 'yes nothing spreads hate like opposing racism &amp; black genocide i agree. tk usrusrusr usrusrusr usrusrusr usrusrusr')
(0.79291036153521355, 'so ending black genocide insights violence? oh really? no black genocide is violent. tk usrusrusr usrusrusr usrusrusr usrusrusr usrusrusr')
(0.79291834637943648, 'yes racists think opposing racism &amp; black genocide makes me a racist. tk usrusrusr usrusrusr usrusrusr usrusrusr usrusrusr')
(0.8036062851138589, 'yes racists call me a monster for opposing black genocide its absurd. tk usrusrusr usrusrusr usrusrusr usrusrusr usrusrusr')
(0.80379103769140281, "mor'on(n) !)someone black destroys the arguments of dumb whites. 2)opposes black genocide. tk usrusrusr usrusrusr usrusrusr usrusrusr")
(0.8129036502182011, 'yes black genocide is anything meant to kill 1.1b blacks. you damn right. tk usrusrusr usrusrusr usrusrusr usrusrusr')
(0.81320378072796051

In [215]:
print("Reverse Frequency of the terms:")
print(i)
for k, v in Counter({int(k):v for k,v in cluster_list[i-1]['rif'].items()}).items():
    if k>1:
        print(k,":",v)

Reverse Frequency of the terms:
1
2 : ['news the', 'indonesia', 'abortion', 'house of', 'oslo', 'language', 'urlurlurl myanmar', 'commemoration', 'rwanda', 'happening', 'genocide against', 'urlurlurl pakistan', 'daily', 'n', 'day', 'in the', 'cry', 'a rohingya', 'don t', 'the rohingya', 'breaking', '4', 'to end', 'the armenian', 'wanted for', 's genocide', 'nazi', 'with', 'part 2', '1', 'breaking news', 'ethnic cleansing', 'voice', 'cleansing of', 'urlurlurl rwanda', 'commemoration of', 'obama s', 'in arakan', 'pacific', 'anniversary', 'now', 'play', 'don', 'urlurlurl obama', 'rohingya language', 'fukushima breaking', 'cultural', 'oslo conference', 'cultural genocide', 'is genocide', 'about', 'armenia', 'urlurlurl what', 'film', 'wanted', 'pakistan', 'who', 'bangladesh', 'ethnic', 'house', '100', 'address', 'conference', 'rohingyas', 'fukushima news', 'myanmar to', 'cover', 'times', 'tv', 'plight', 'mark', 'speaks', 'of genocide', 'burma times', 'srebrenica genocide', 'their', 'genocid

In [252]:
rlvdb2, rlvcl2 = rlv.connect_mongodb(configfile='ebasar2.ini',coll_name="testcl")

collection_name = 'genocide_clusters_20151005'
rlvdb2[collection_name].insert(cluster_list) #Each iteration results with a candidate cluster list. Each iteration will have its own list. Therefore they are not mixed.
print("Clusters were written to the collection:", collection_name)

OperationFailure: quota exceeded

In [ ]:
import json
with open('data.txt', 'w') as outfile:
    json.dump(cluster_list, "genocide_clusters.json")

In [ ]:
x = 5
x

In [ ]:
# import pickle
# with open("genocide_clusters_20151005", 'wb') as f:
#     pickle.dump(cluster_list, f, pickle.HIGHEST_PROTOCOL)
#     print("wrote km user to pickle")

In [3]:
import pickle
with open("genocide_clusters_20151005", 'rb') as f:
    cluster_list = pickle.load(f)

read km_user from pickle


In [4]:
cluster_list[0].keys()

dict_keys(['cno', 'twids', 'cnoprefix', 'cstr', 'user_entropy', 'ctweettuplelist', '_id', 'rif'])

In [249]:
cluster_list_small = []
for c in cluster_list:
    cluster_list_small.append({k:v for k,v in c.items() if k in ["cno","cstr","user_entropy","rif","_id","cnoprefix",'twids']})

In [250]:


collection_name = 'genocide_clusters_small_20151005'
rlvdb[collection_name].insert(cluster_list_small) #Each iteration results with a candidate cluster list. Each iteration will have its own list. Therefore they are not mixed.
print("Clusters were written to the collection:", collection_name)

OperationFailure: quota exceeded